# Automate the Boring Stuff - EMAILS!

[PyzMail Package Doku](http://magiksys.net/pyzmail/)  
[ImapClient Package Doku](https://imapclient.readthedocs.org)  
[Automate the Boring Stuff](https://automatetheboringstuff.com/chapter16)  

In [ ]:
!pip install imapclient
# !pip install pyzmail # funktioniert nicht!!!
!pip install pyzmail36 

### Imports

In [1]:
import pandas as pd
import imapclient
import pyzmail

In [2]:
HOST = 'imap.web.de'
ADRESS = input("Emailadresse:")
PW = input("Passwort:")

conn = imapclient.IMAPClient(HOST, ssl=True)
conn.login(ADRESS, PW)

Emailadresse: barth.andreas@web.de
Passwort: sauron


b'LOGIN completed'

In [3]:
folder = 'Mailtest' # 'INBOX'

conn.select_folder(folder, readonly=True)

{b'PERMANENTFLAGS': (b'\\Answered',
  b'\\Flagged',
  b'\\Draft',
  b'\\Deleted',
  b'\\Seen',
  b'NonJunk',
  b'\\*'),
 b'EXISTS': 13,
 b'RECENT': 0,
 b'FLAGS': (b'\\Answered',
  b'\\Flagged',
  b'\\Deleted',
  b'\\Seen',
  b'\\Draft',
  b'NonJunk'),
 b'UIDNEXT': 14,
 b'UIDVALIDITY': 1591783832,
 b'READ-ONLY': [b'']}

[Enum IMAPClient.SEARCH_CRITERIA](https://commons.apache.org/proper/commons-net/javadocs/api-3.6/org/apache/commons/net/imap/IMAPClient.SEARCH_CRITERIA.html)

[IMAP Search Strings](http://www.marshallsoft.com/ImapSearch.htm)

In [4]:
UIDS = conn.search(['ALL'])
# UIDS = conn.search(['BEFORE-05-06-2020'])

UIDS

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [5]:
dir(pyzmail)

['PyzMessage',
 'PzMessage',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'compose_mail',
 'decode_text',
 'email_address_re',
 'generate',
 'message_from_binary_file',
 'message_from_bytes',
 'message_from_file',
 'message_from_string',
 'parse',
 'send_mail',
 'send_mail2',
 'utils',
 'version']

In [6]:
emailsDF = pd.DataFrame(columns=["FROM", "TO", "CC", "BCC", "SUB", "TXT"])
for messageKey in UIDS:
    # Get Message
    rawMessage = conn.fetch(messageKey, ['BODY[]', 'FLAGS'] )
    message = pyzmail.PyzMessage.factory(rawMessage[messageKey] [b'BODY[]'])
    
    # From / To / CC / BCC / Subject auslesen
    sentFROM = message.get_addresses('from')
    sentTO = message.get_addresses('to')
    sentCC = message.get_addresses('cc')
    sentBCC = message.get_addresses('bcc')
    SUBJECT = message.get_subject()
    
    # Email Textbody auslesen
    messageBody = message.text_part.get_payload().decode('UTF-8')
    messageBodyClean = messageBody.replace("\r\n", " <NL> ")
    
    emailsDF.loc[messageKey,:] = [sentFROM, sentTO, sentCC, sentBCC, SUBJECT, messageBodyClean]
   

AttributeError: 'NoneType' object has no attribute 'get_payload'

In [22]:
emailsDF

,FROM,TO,CC,BCC,SUB,TXT
155,"[(Aboalarm GmbH, receipts+k6Blo61skcL6x0thb2Cy...","[(barth.andreas@web.de, barth.andreas@web.de)]",[],[],Your Aboalarm GmbH receipt [#1912-5086],<NL> <NL> <NL> Thanks for your €1.49 paymen...
159,"[(Central Krankenversicherung, info@central.de)]","[(barth.andreas@web.de, barth.andreas@web.de)]",[],[],Ihr Dialog mit der Central,"Sehr geehrte Damen und Herren, <NL> <NL> Ihr ..."
167,"[(www.peterkleine.com, mail@peterkleine.com)]","[(Andreas Barth, barth.andreas@web.de)]",[],[],Mail vom 13.12.2015,"<NL> Moin Andreas! <NL> Ich hoffe, es geht Di..."
172,"[(Andreas Barth, barth.andreas@web.de)]","[(customer-service@oanda.co.uk, customer-servi...",[],[],Change of Adress - Primary 538070,"Dear Sirs and Madams, <NL> <NL> <NL> <NL>..."


In [24]:
emailsDF.loc[167,"TXT"]

' <NL> Moin Andreas! <NL> Ich hoffe, es geht Dir gut und die OP ist gut verlaufen? <NL> Wie versprochen sende ich Dir mal ein paar Infos über unser kleines  <NL> Start-up! <NL>  <NL> Wäre gut, wenn das nicht jeder lesen würde, okay? <NL>  <NL> Wir schnacken! <NL>  <NL> LG, <NL> P. <NL>  <NL>  <NL> Keyfacts: <NL> pellybay gmbh <NL> An der Alster 18 <NL> 20099 Hamburg <NL> Gründung 8.1.2013 aus der GmbH "Boutique de Films" <NL> Neueröffnung am 15.5.2013 <NL> Angestellte: <NL> Philip, Head of Production <NL> Marie, PA <NL> Lars, PA <NL> Gesellschafter: Seydlitz und Kleine zu je 50%. <NL> GF: Seydlitz (nicht aktiv), Bothe (nicht aktiv), Kleine (richtig aktiv) <NL>  <NL>  <NL> Warum gibt es pellybay? <NL> 1. Bewegtbild für Unternehmen wird es immer geben <NL> 2. Unternehmen benötigen in immer kürzeren Abständen gutes Bewegtbild <NL> 3. Bewegtbild, Umsetzung und Endfertigung liegen immer mehr in einer Hand,  <NL> da Technik und Software günstig sind. <NL> Jeder kann technisch zwar alles mach

Abgreifen einzelner Informationen einer Message

In [ ]:
messageKey = 167
rawMessage = conn.fetch(messageKey, ['BODY[]', 'FLAGS'] )

In [ ]:
message = pyzmail.PyzMessage.factory(rawMessage[messageKey] [b'BODY[]'])

In [ ]:
message.get_subject()

In [ ]:
message.get_addresses('from')

In [ ]:
message.get_addresses('to')

In [ ]:
message.get_addresses('cc')

In [ ]:
message.text_part

In [15]:
message.

AttributeError: 'PyzMessage' object has no attribute 'get_date'

In [ ]:
message.html_part

In [ ]:
message.html_part == None

In [ ]:
messageBody = message.text_part.get_payload().decode('UTF-8')
messageBody = messageBody.replace("\r\n", " <NL> ")
messageBody

In [ ]:
conn.list_folders()  # 3. Tupel pro Zeile ist der entsprechende Ordner

In [ ]:
import pandas as pd

In [ ]:
UIDS

In [ ]:
emailsDF = pd.DataFrame(columns=["FROM", "TO", "CC", "BCC", "SUB", "TXT"])
emailsDF

In [ ]:
emailsDF = pd.DataFrame(columns=["FROM", "TO", "CC", "BCC", "SUB", "TXT"])
for messageKey in UIDS:
    # Get Message
    rawMessage = conn.fetch(messageKey, ['BODY[]', 'FLAGS'] )
    message = pyzmail.PyzMessage.factory(rawMessage[messageKey] [b'BODY[]'])
    
    # From / To / CC / BCC / Subject auslesen
    sentFROM = message.get_addresses('from')
    sentTO = message.get_addresses('to')
    sentCC = message.get_addresses('cc')
    sentBCC = message.get_addresses('bcc')
    SUBJECT = message.get_subject()
    
    # Email Textbody auslesen
    messageBody = message.text_part.get_payload().decode('UTF-8')
    messageBody = messageBody.replace("\r\n", " <NL> ")
    
    emailsDF[messageKey] = [sentFROM, sentTO, sentCC, sentBCC, SUBJECT, messageBody]
    
    
    